In [ ]:
import uuid

import boto3
from dotenv import load_dotenv

from bring_a_crew_bedrock.inline_agent_utils import invoke_inline_agent_helper, load_json_file

_ = load_dotenv()

region = "eu-west-1"
recreate_lambdas = False

# Runtime Endpoints
bedrock_rt_client = boto3.client(
    "bedrock-agent-runtime",
    region_name=region
)

In [ ]:
session_id = str(uuid.uuid4())
model_id = "eu.amazon.nova-lite-v1:0"
instruction = (
    "You are a scheduling assistant. You help in checking the availability of people. Do not make up "
    "availability if you do not know the person's schedule. Answer in short sentences, stick to the "
    "answer to the question."
)

request_params = {
    "enableTrace": True,
    "endSession": False,
    "foundationModel": model_id,
    "instruction": instruction,
    "sessionId": session_id
}

In [ ]:
request_params["inputText"] = "Can you help me schedule an appointment with Bob?"

invoke_inline_agent_helper(bedrock_rt_client, request_params, trace_level="core")

In [ ]:
# run lambda function creation
from lambda_creator import create_lambda_function_and_its_resources, remove_lambda_function_and_its_resources
import os

sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()["Account"]

present_directory = os.getcwd()
lambda_function_code_path = str(present_directory) + "/lambdas/lambda_function_schedule.py"

# Create all resources
resources = create_lambda_function_and_its_resources(
    region=region,
    account_id=account_id,
    custom_name=f"inlineagent-scheduler",
    lambda_code_path=lambda_function_code_path
)

# Access the created resources
lambda_function = resources['lambda_function']
lambda_function_arn = lambda_function['FunctionArn']
print(lambda_function_arn)


In [ ]:
present_directory = os.getcwd()
payload_schedule_path = str(present_directory) + "/lambdas/payload-schedule.json"

# Load the OpenAPI schema as a json file
fetch_schedule_payload = load_json_file(payload_schedule_path)

apply_schedule_tool = {
    "actionGroupName": "FetchSchedule",
    "actionGroupExecutor": {
        "lambda": lambda_function_arn
    },
    "apiSchema": {
        "payload": fetch_schedule_payload
    },
    "description": "Find availability of a person"
}

request_params["actionGroups"] = [apply_schedule_tool]

In [ ]:
request_params["inputText"] = "Schedule a meeting with Bob after 2025-04-07"

invoke_inline_agent_helper(bedrock_rt_client, request_params, trace_level="core")

In [ ]:
request_params[
    "inputText"] = "Yes, please schedule the meeting for the thursday in the morning after the 7th of April 2025"

invoke_inline_agent_helper(bedrock_rt_client, request_params, trace_level="core")

In [ ]:
# Warning only run this cell when you are done ...

remove_lambda_function_and_its_resources(
    region=region,
    account_id=account_id,
    custom_name=f"inlineagent-scheduler",
)